# GraphKT M3 Final: Mastery Features + Weights + Dual Propagation

**Architecture:**
- Student features: [5D stats + 397D mastery vector] = 402D (from M3a)
- Dual propagation: Structural (Q↔T↔C) + Knowledge (S↔C weighted) (from M3b)
- Mastery in both identity and edge weights
- Update: Epoch-level for features, batch-level for weights

**Expected: ~77% val, ~76.5% test (+1% vs M3b)**

In [ ]:
# ============================================================
# CELL 1: CONFIGURATION
# ============================================================

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import RGCNConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')

class Config:
    """Configuration for GraphKT M3 Final"""
    
    # Model
    MODEL_NAME = "GraphKT M3 Final (Features + Weights + Dual Propagation)"
    EMBED_DIM = 32
    HIDDEN_DIM = 64
    NUM_RGCN_LAYERS = 2
    
    # Training
    LEARNING_RATE = 1e-4
    BATCH_SIZE = 512
    MAX_EPOCHS = 100
    PATIENCE = 10
    
    # Mastery
    MASTERY_INIT = 0.5
    MASTERY_LR = 0.01
    
    # Data
    TEST_STUDENT_RATIO = 0.15
    NUM_FOLDS = 5
    SEED = 42
    
    # Device
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

config = Config()

# Set seeds
np.random.seed(config.SEED)
torch.manual_seed(config.SEED)
torch.cuda.manual_seed_all(config.SEED)

print(f"Configuration: {config.MODEL_NAME}")
print(f"Device: {config.DEVICE}")
print(f"Embed dim: {config.EMBED_DIM}, Hidden dim: {config.HIDDEN_DIM}")
print(f"Mastery init: {config.MASTERY_INIT}, Mastery LR: {config.MASTERY_LR}")

In [ ]:
# ============================================================
# CELL 2: LOAD DATASET
# ============================================================

print("Loading Algebra 2005-2006 dataset...")
df = pd.read_csv('algebra_2005_2006_train.txt', delimiter='\t')

print(f"\nRaw data shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

In [ ]:
# ============================================================
# CELL 3: DATA CLEANING AND PREPROCESSING
# ============================================================

print("Data cleaning and preprocessing...\n")

# Rename columns
df = df.rename(columns={
    'Anon Student Id': 'student_id',
    'Problem Name': 'question_id', 
    'Step Name': 'step_id',
    'KC(Default)': 'kc_id',
    'Correct First Attempt': 'correct',
    'Incorrects': 'incorrects',
    'Hints': 'hints',
    'Correct Step Duration (sec)': 'duration'
})

# Remove rows with missing KCs
df = df.dropna(subset=['kc_id'])

# Handle multiple KCs (take first)
df['kc_id'] = df['kc_id'].apply(lambda x: x.split('~~')[0] if isinstance(x, str) else x)

# Convert correct to binary
df['correct'] = df['correct'].astype(int)

# Create indices
df['student_idx'] = pd.Categorical(df['student_id']).codes
df['question_idx'] = pd.Categorical(df['question_id']).codes
df['step_idx'] = pd.Categorical(df['step_id']).codes
df['kc_idx'] = pd.Categorical(df['kc_id']).codes

# Process duration
df['duration'] = df['duration'].fillna(df['duration'].median())
df['log_duration'] = np.log1p(df['duration'])

# Compute aggregated features
question_stats = df.groupby('question_idx').agg({
    'correct': 'mean',
    'log_duration': 'mean',
    'hints': 'mean',
    'incorrects': 'mean'
}).reset_index()
question_stats.columns = ['question_idx', 'question_difficulty', 
                         'avg_log_duration', 'avg_hints', 'avg_incorrects']

step_stats = df.groupby('step_idx').agg({
    'correct': 'mean'
}).reset_index()
step_stats.columns = ['step_idx', 'step_difficulty']

df = df.merge(question_stats, on='question_idx', how='left')
df = df.merge(step_stats, on='step_idx', how='left')

# Get counts
num_students = df['student_idx'].nunique()
num_questions = df['question_idx'].nunique()
num_steps = df['step_idx'].nunique()
num_kcs = df['kc_idx'].nunique()

print(f"After cleaning:")
print(f"  Total students: {num_students}")
print(f"  Total questions: {num_questions}")
print(f"  Total steps: {num_steps}")
print(f"  Total KCs: {num_kcs}")
print(f"  Total interactions: {len(df)}")
print(f"  Success rate: {df['correct'].mean():.2%}")

In [ ]:
# ============================================================
# CELL 4: STUDENT-LEVEL SPLIT (5-FOLD + TEST)
# ============================================================

print("Creating student-level train/test split...\n")

all_students = df['student_idx'].unique()
np.random.shuffle(all_students)

# Split: 85% train/val, 15% test
non_test_students, test_students = train_test_split(
    all_students,
    test_size=config.TEST_STUDENT_RATIO,
    random_state=config.SEED
)

df_non_test = df[df['student_idx'].isin(non_test_students)].copy()
df_test = df[df['student_idx'].isin(test_students)].copy()

print(f"TEST set (held out):")
print(f"  Students: {len(test_students)} ({len(test_students)/len(all_students)*100:.1f}%)")
print(f"  Interactions: {len(df_test):,}")

# Create 5 folds from non-test students
np.random.shuffle(non_test_students)
fold_size = len(non_test_students) // config.NUM_FOLDS

fold_assignments = {}
for fold in range(config.NUM_FOLDS):
    start_idx = fold * fold_size
    end_idx = start_idx + fold_size if fold < config.NUM_FOLDS - 1 else len(non_test_students)
    
    val_students = non_test_students[start_idx:end_idx]
    train_students = np.concatenate([
        non_test_students[:start_idx],
        non_test_students[end_idx:]
    ])
    
    fold_assignments[fold] = {
        'train_students': train_students,
        'val_students': val_students
    }
    
    df_val = df_non_test[df_non_test['student_idx'].isin(val_students)]
    print(f"\nFold {fold+1}:")
    print(f"  Train students: {len(train_students)}")
    print(f"  Val students: {len(val_students)}")
    print(f"  Val interactions: {len(df_val):,}")

print(f"\n{'='*60}")
print(f"Non-test (enters K-Fold):")
print(f"  Students: {len(non_test_students)} ({len(non_test_students)/len(all_students)*100:.1f}%)")
print(f"  Interactions: {len(df_non_test):,}")
print(f"\nReady for 5-Fold Cross-Validation!")
print(f"{'='*60}")

In [ ]:
# ============================================================
# CELL 5: M3 FINAL ARCHITECTURE
# Combines: Mastery Features (M3a) + Mastery Weights + Dual Propagation (M3b)
# ============================================================

def compute_node_features_with_mastery(df_interactions, num_students, num_questions, 
                                       num_steps, num_kcs, mastery_matrix):
    """
    Compute node features WITH mastery for students (from M3a).
    
    Students: [5D statistical features + mastery_vector] = (5 + num_kcs)D
    Others (Q, T, C): 5D statistical features + zero-padding
    
    Args:
        mastery_matrix: [num_students, num_kcs] tensor
        
    Returns:
        features: [total_nodes, 5 + num_kcs] tensor
    """
    feature_dim = 5 + num_kcs  # 5 + 397 = 402 for Algebra dataset
    total_nodes = num_students + num_questions + num_steps + num_kcs
    features = torch.zeros(total_nodes, feature_dim, dtype=torch.float32)
    
    # STUDENTS: [5D stats + mastery]
    for student_id in range(num_students):
        student_data = df_interactions[df_interactions['student_idx'] == student_id]
        
        if len(student_data) > 0:
            # 5D statistical features
            stats_5d = torch.tensor([
                np.log1p(len(student_data)),
                student_data['correct'].mean(),
                student_data['avg_log_duration'].iloc[0],
                student_data['avg_hints'].iloc[0],
                student_data['avg_incorrects'].iloc[0]
            ], dtype=torch.float32)
            
            # Concatenate [5D + mastery_397D]
            features[student_id] = torch.cat([stats_5d, mastery_matrix[student_id]])
    
    # QUESTIONS: 5D features only (rest zeros)
    offset_q = num_students
    for q_id in range(num_questions):
        q_data = df_interactions[df_interactions['question_idx'] == q_id]
        if len(q_data) > 0:
            features[offset_q + q_id, :5] = torch.tensor([
                np.log1p(len(q_data)),
                q_data['correct'].mean(),
                q_data['question_difficulty'].iloc[0],
                q_data['avg_hints'].mean(),
                q_data['avg_incorrects'].mean()
            ], dtype=torch.float32)
    
    # STEPS: 5D features only
    offset_s = num_students + num_questions
    for step_id in range(num_steps):
        step_data = df_interactions[df_interactions['step_idx'] == step_id]
        if len(step_data) > 0:
            features[offset_s + step_id, :5] = torch.tensor([
                np.log1p(len(step_data)),
                step_data['correct'].mean(),
                step_data['step_difficulty'].iloc[0],
                step_data['avg_hints'].mean(),
                step_data['avg_incorrects'].mean()
            ], dtype=torch.float32)
    
    # KCS: 5D features only
    offset_c = num_students + num_questions + num_steps
    for kc_id in range(num_kcs):
        kc_data = df_interactions[df_interactions['kc_idx'] == kc_id]
        if len(kc_data) > 0:
            features[offset_c + kc_id, :5] = torch.tensor([
                np.log1p(len(kc_data)),
                kc_data['correct'].mean(),
                0.5,  # No specific difficulty for KC
                kc_data['avg_hints'].mean(),
                kc_data['avg_incorrects'].mean()
            ], dtype=torch.float32)
    
    return features


def build_heterogeneous_graph(df_interactions, num_students, num_questions, 
                             num_steps, num_kcs):
    """
    Build heterogeneous graph with 6 relation types.
    Returns edge_index dict and relation mapping.
    """
    offset_q = num_students
    offset_s = num_students + num_questions  
    offset_c = num_students + num_questions + num_steps
    
    edges = {
        'student_question': [],
        'question_student': [],
        'question_step': [],
        'step_question': [],
        'step_kc': [],
        'kc_step': []
    }
    
    # S ↔ Q edges
    for _, row in df_interactions[['student_idx', 'question_idx']].drop_duplicates().iterrows():
        s, q = int(row['student_idx']), int(row['question_idx'])
        edges['student_question'].append([s, offset_q + q])
        edges['question_student'].append([offset_q + q, s])
    
    # Q ↔ T edges
    for _, row in df_interactions[['question_idx', 'step_idx']].drop_duplicates().iterrows():
        q, t = int(row['question_idx']), int(row['step_idx'])
        edges['question_step'].append([offset_q + q, offset_s + t])
        edges['step_question'].append([offset_s + t, offset_q + q])
    
    # T ↔ C edges
    for _, row in df_interactions[['step_idx', 'kc_idx']].drop_duplicates().iterrows():
        t, c = int(row['step_idx']), int(row['kc_idx'])
        edges['step_kc'].append([offset_s + t, offset_c + c])
        edges['kc_step'].append([offset_c + c, offset_s + t])
    
    # Convert to tensors
    edge_index_dict = {}
    for rel_type, edge_list in edges.items():
        if len(edge_list) > 0:
            edge_index_dict[rel_type] = torch.tensor(edge_list, dtype=torch.long).t()
    
    all_relations = ['student_question', 'question_student', 'question_step',
                    'step_question', 'step_kc', 'kc_step']
    
    return edge_index_dict, all_relations


class StructuralMessagePassing(nn.Module):
    """Structural propagation: Q ↔ T ↔ C (uniform weighting)"""
    def __init__(self, hidden_dim, num_relations):
        super().__init__()
        self.rgcn = RGCNConv(hidden_dim, hidden_dim, num_relations)
    
    def forward(self, x, edge_index, edge_type):
        return self.rgcn(x, edge_index, edge_type)


class KnowledgeMessagePassing(nn.Module):
    """Knowledge propagation: S ↔ C (mastery-weighted)"""
    def __init__(self, hidden_dim):
        super().__init__()
        self.transform = nn.Linear(hidden_dim, hidden_dim)
    
    def forward(self, x, edge_index, mastery_weights):
        src, dst = edge_index
        messages = self.transform(x[src]) * mastery_weights.unsqueeze(1)
        out = torch.zeros_like(x)
        out.index_add_(0, dst, messages)
        return out


class RGCNWithDualPropagation(nn.Module):
    """
    M3 Final Architecture:
    - Input: [5D + mastery] for students, 5D for others
    - Dual propagation: Structural (Q↔T↔C) + Knowledge (S↔C weighted)
    - Fusion: Concat both pathways
    """
    def __init__(self, input_dim, hidden_dim, num_relations, num_kcs):
        super().__init__()
        
        # Encoders (input_dim = 5 + num_kcs for students)
        self.student_encoder = nn.Linear(input_dim, hidden_dim)
        self.other_encoder = nn.Linear(input_dim, hidden_dim)
        
        # Structural propagation (Q↔T↔C)
        self.structural_prop = StructuralMessagePassing(hidden_dim, num_relations)
        
        # Knowledge propagation (S↔C weighted)
        self.knowledge_prop = KnowledgeMessagePassing(hidden_dim)
        
        # Fusion
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Prediction head
        self.predictor = nn.Sequential(
            nn.Linear(hidden_dim * 3, hidden_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, x, edge_index_dict, all_relations, mastery_matrix,
                num_students, num_questions, num_steps, num_kcs):
        """
        Args:
            x: Node features [N, input_dim]
            edge_index_dict: Dict of edge indices per relation
            all_relations: List of relation names
            mastery_matrix: [num_students, num_kcs]
        """
        # Encode (different encoders for students vs others)
        h = torch.zeros(x.size(0), self.student_encoder.out_features, device=x.device)
        h[:num_students] = F.relu(self.student_encoder(x[:num_students]))
        h[num_students:] = F.relu(self.other_encoder(x[num_students:]))
        
        # PATH 1: Structural propagation (Q↔T↔C)
        structural_edges = []
        structural_types = []
        for i, rel in enumerate(['question_step', 'step_question', 'step_kc', 'kc_step']):
            if rel in edge_index_dict:
                structural_edges.append(edge_index_dict[rel])
                structural_types.append(torch.full((edge_index_dict[rel].size(1),), i, dtype=torch.long))
        
        if len(structural_edges) > 0:
            struct_edge_index = torch.cat(structural_edges, dim=1).to(x.device)
            struct_edge_type = torch.cat(structural_types).to(x.device)
            h_structural = self.structural_prop(h, struct_edge_index, struct_edge_type)
        else:
            h_structural = h
        
        # PATH 2: Knowledge propagation (S↔C weighted by mastery)
        offset_c = num_students + num_questions + num_steps
        
        # S → C edges
        s_to_c_edges = []
        s_to_c_weights = []
        for s in range(num_students):
            for c in range(num_kcs):
                s_to_c_edges.append([s, offset_c + c])
                s_to_c_weights.append(mastery_matrix[s, c].item())
        
        if len(s_to_c_edges) > 0:
            s_to_c_index = torch.tensor(s_to_c_edges, dtype=torch.long).t().to(x.device)
            s_to_c_weights = torch.tensor(s_to_c_weights, dtype=torch.float32).to(x.device)
            h_knowledge = self.knowledge_prop(h, s_to_c_index, s_to_c_weights)
        else:
            h_knowledge = h
        
        # FUSION: Concat both pathways
        h_fused = self.fusion(torch.cat([h_structural, h_knowledge], dim=1))
        
        return h_fused
    
    def predict(self, h_fused, student_ids, step_ids, kc_ids, 
                num_students, num_questions, num_steps):
        """Predict from fused embeddings"""
        offset_s = num_students + num_questions
        offset_c = num_students + num_questions + num_steps
        
        h_students = h_fused[student_ids]
        h_steps = h_fused[offset_s + step_ids]
        h_kcs = h_fused[offset_c + kc_ids]
        
        combined = torch.cat([h_students, h_steps, h_kcs], dim=1)
        logits = self.predictor(combined).squeeze()
        return logits


def update_mastery(mastery_matrix, student_ids, kc_ids, outcomes, lr=0.01):
    """Update mastery based on batch outcomes"""
    for s, c, y in zip(student_ids, kc_ids, outcomes):
        current = mastery_matrix[s, c].item()
        delta = lr * (y - current)
        mastery_matrix[s, c] = torch.clamp(
            mastery_matrix[s, c] + delta, 0.0, 1.0
        )


def train_epoch(model, optimizer, criterion, train_loader, mastery_matrix, 
               node_features, edge_index_dict, all_relations,
               num_students, num_questions, num_steps, num_kcs, device):
    """Training loop for one epoch"""
    model.train()
    total_loss = 0
    
    for batch in train_loader:
        student_ids = batch['student_id'].to(device)
        step_ids = batch['step_id'].to(device)
        kc_ids = batch['kc_id'].to(device)
        labels = batch['label'].float().to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        h_fused = model(node_features, edge_index_dict, all_relations,
                       mastery_matrix, num_students, num_questions, num_steps, num_kcs)
        
        logits = model.predict(h_fused, student_ids, step_ids, kc_ids,
                              num_students, num_questions, num_steps)
        
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        # Update mastery
        with torch.no_grad():
            preds = torch.sigmoid(logits)
            update_mastery(mastery_matrix, student_ids.cpu(), kc_ids.cpu(),
                          labels.cpu(), lr=config.MASTERY_LR)
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)


@torch.no_grad()
def evaluate(model, data_loader, mastery_matrix, node_features, edge_index_dict,
            all_relations, num_students, num_questions, num_steps, num_kcs, device):
    """Evaluation loop"""
    model.eval()
    all_preds = []
    all_labels = []
    
    for batch in data_loader:
        student_ids = batch['student_id'].to(device)
        step_ids = batch['step_id'].to(device)
        kc_ids = batch['kc_id'].to(device)
        labels = batch['label'].float().to(device)
        
        h_fused = model(node_features, edge_index_dict, all_relations,
                       mastery_matrix, num_students, num_questions, num_steps, num_kcs)
        
        logits = model.predict(h_fused, student_ids, step_ids, kc_ids,
                              num_students, num_questions, num_steps)
        
        preds = torch.sigmoid(logits)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    auc = roc_auc_score(all_labels, all_preds)
    acc = accuracy_score(all_labels, np.array(all_preds) > 0.5)
    
    return auc, acc


def create_data_loader(df, batch_size, shuffle=True):
    """Create PyTorch data loader"""
    dataset = []
    for _, row in df.iterrows():
        dataset.append({
            'student_id': int(row['student_idx']),
            'step_id': int(row['step_idx']),
            'kc_id': int(row['kc_idx']),
            'label': int(row['correct'])
        })
    
    return torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle,
        collate_fn=lambda x: {k: torch.tensor([d[k] for d in x]) for k in x[0].keys()}
    )


def run_single_fold(fold, fold_assignments, df_non_test, 
                   num_students, num_questions, num_steps, num_kcs):
    """
    Train and evaluate M3 Final on a single fold.
    
    Key steps:
    1. Initialize mastery BEFORE computing features
    2. Compute features WITH mastery [5D + 397D]
    3. Build graph
    4. Each epoch: RECALCULATE features with updated mastery
    5. Train with dual propagation + mastery weights
    """
    device = torch.device(config.DEVICE)
    print(f"\n{'='*60}")
    print(f"FOLD {fold+1}/{config.NUM_FOLDS}")
    print(f"{'='*60}")
    
    # Get train/val split
    train_students = fold_assignments[fold]['train_students']
    val_students = fold_assignments[fold]['val_students']
    
    df_train = df_non_test[df_non_test['student_idx'].isin(train_students)].copy()
    df_val = df_non_test[df_non_test['student_idx'].isin(val_students)].copy()
    
    print(f"Train: {len(train_students)} students, {len(df_train):,} interactions")
    print(f"Val: {len(val_students)} students, {len(df_val):,} interactions")
    
    # STEP 1: Initialize mastery FIRST
    mastery_matrix = torch.full(
        (num_students, num_kcs), 
        config.MASTERY_INIT, 
        dtype=torch.float32, 
        device=device
    )
    
    # STEP 2: Compute node features WITH mastery (402D for students)
    node_features = compute_node_features_with_mastery(
        df_non_test, num_students, num_questions, num_steps, num_kcs,
        mastery_matrix
    ).to(device)
    
    print(f"\nNode features: {node_features.shape} (students are 402D)")
    
    # Build graph
    edge_index_dict, all_relations = build_heterogeneous_graph(
        df_non_test, num_students, num_questions, num_steps, num_kcs
    )
    
    print(f"Graph relations: {len(all_relations)}")
    for rel in all_relations:
        if rel in edge_index_dict:
            print(f"  {rel}: {edge_index_dict[rel].size(1)} edges")
    
    # Create data loaders
    train_loader = create_data_loader(df_train, config.BATCH_SIZE, shuffle=True)
    val_loader = create_data_loader(df_val, config.BATCH_SIZE, shuffle=False)
    
    # Initialize model (input_dim = 5 + num_kcs)
    model = RGCNWithDualPropagation(
        input_dim=5 + num_kcs,  # 402D for Algebra
        hidden_dim=config.HIDDEN_DIM,
        num_relations=len(all_relations),
        num_kcs=num_kcs
    ).to(device)
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"\nModel parameters: {total_params:,}")
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    criterion = nn.BCEWithLogitsLoss()
    
    # Training loop
    best_val_auc = 0
    patience_counter = 0
    
    print(f"\nTraining...")
    for epoch in range(config.MAX_EPOCHS):
        # CRITICAL: Recalculate features with updated mastery each epoch
        node_features = compute_node_features_with_mastery(
            df_non_test, num_students, num_questions, num_steps, num_kcs,
            mastery_matrix
        ).to(device)
        
        train_loss = train_epoch(
            model, optimizer, criterion, train_loader, mastery_matrix,
            node_features, edge_index_dict, all_relations,
            num_students, num_questions, num_steps, num_kcs, device
        )
        
        val_auc, val_acc = evaluate(
            model, val_loader, mastery_matrix, node_features, edge_index_dict,
            all_relations, num_students, num_questions, num_steps, num_kcs, device
        )
        
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"Epoch {epoch+1:3d}: Loss={train_loss:.4f}, "
                  f"Val AUC={val_auc:.4f}, Val Acc={val_acc:.4f}")
        
        # Early stopping
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
            if patience_counter >= config.PATIENCE:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    # Final evaluation with best model
    model.load_state_dict(best_model_state)
    val_auc, val_acc = evaluate(
        model, val_loader, mastery_matrix, node_features, edge_index_dict,
        all_relations, num_students, num_questions, num_steps, num_kcs, device
    )
    
    print(f"\n{'='*60}")
    print(f"Fold {fold+1} Results:")
    print(f"  Best Val AUC: {val_auc:.4f}")
    print(f"  Val Accuracy: {val_acc:.4f}")
    print(f"  Epochs trained: {epoch+1}")
    print(f"{'='*60}")
    
    return {
        'val_auc': val_auc,
        'val_acc': val_acc,
        'epochs': epoch + 1,
        'num_students': len(val_students),
        'num_interactions': len(df_val),
        'model_state': best_model_state,
        'mastery_matrix': mastery_matrix.cpu()
    }

print("\n✅ M3 Final architecture defined!")
print("Key features:")
print("  - Student features: [5D + 397D mastery] = 402D")
print("  - Dual propagation: Structural (Q↔T↔C) + Knowledge (S↔C weighted)")
print("  - Mastery in both features AND weights")
print("  - Dynamic feature updates each epoch")

In [ ]:
# ============================================================
# CELL 6: 5-FOLD CROSS-VALIDATION
# ============================================================

print("\n" + "="*60)
print("STARTING 5-FOLD CROSS-VALIDATION")
print("="*60)

fold_results = []

for fold in range(config.NUM_FOLDS):
    result = run_single_fold(
        fold=fold,
        fold_assignments=fold_assignments,
        df_non_test=df_non_test,
        num_students=num_students,
        num_questions=num_questions,
        num_steps=num_steps,
        num_kcs=num_kcs
    )
    fold_results.append(result)

# Summary statistics
val_aucs = [r['val_auc'] for r in fold_results]
val_accs = [r['val_acc'] for r in fold_results]
epochs_list = [r['epochs'] for r in fold_results]

print("\n" + "="*60)
print("5-FOLD CROSS-VALIDATION SUMMARY")
print("="*60)
print(f"\nValidation AUC: {np.mean(val_aucs):.4f} ± {np.std(val_aucs):.4f}")
print(f"Validation Accuracy: {np.mean(val_accs):.4f} ± {np.std(val_accs):.4f}")
print(f"Average epochs: {np.mean(epochs_list):.1f}")

print("\n" + "="*60)
print("DETAILED RESULTS FOR PAPER")
print("="*60)

print("\nTable 1: Per-Fold Cross-Validation Results")
print("-" * 55)
print(f"{'Fold':<6} {'Students':<10} {'Interactions':<14} {'Val AUC':<10} {'Val Acc':<10} {'Epochs':<8}")
print("-" * 55)
for i, result in enumerate(fold_results):
    print(f"{i+1:<6} {result['num_students']:<10} {result['num_interactions']:<14,} "
          f"{result['val_auc']:<10.4f} {result['val_acc']:<10.4f} {result['epochs']:<8}")
print("-" * 55)
print(f"{'Mean':<6} {'':<10} {'':<14} {np.mean(val_aucs):<10.4f} "
      f"{np.mean(val_accs):<10.4f} {np.mean(epochs_list):<8.1f}")
print(f"{'±Std':<6} {'':<10} {'':<14} {np.std(val_aucs):<10.4f} "
      f"{np.std(val_accs):<10.4f} {np.std(epochs_list):<8.1f}")
print("-" * 55)

# Calculate total parameters (from first fold)
device = torch.device(config.DEVICE)
model = RGCNWithDualPropagation(
    input_dim=5 + num_kcs,
    hidden_dim=config.HIDDEN_DIM,
    num_relations=6,
    num_kcs=num_kcs
).to(device)
total_params = sum(p.numel() for p in model.parameters())

print("\nConsistency Analysis:")
auc_range = max(val_aucs) - min(val_aucs)
cv = (np.std(val_aucs) / np.mean(val_aucs)) * 100
print(f"  AUC range across folds: {auc_range:.4f}")
if auc_range < 0.03:
    print(f"  ✓ Highly consistent (range < 0.03)")
elif auc_range < 0.05:
    print(f"  ✓ Reasonably consistent (range < 0.05)")
else:
    print(f"  ⚠ High variance (range ≥ 0.05)")
print(f"  Coefficient of variation: {cv:.2f}%")

# Store results for test evaluation
cv_results = {
    'fold_results': fold_results,
    'mean_val_auc': np.mean(val_aucs),
    'std_val_auc': np.std(val_aucs),
    'mean_val_acc': np.mean(val_accs),
    'total_params': total_params
}

print("\n✅ Cross-validation complete!")

In [ ]:
# ============================================================
# CELL 7: TEST SET EVALUATION
# ============================================================

print("\n" + "="*60)
print("EVALUATING ON HELD-OUT TEST SET")
print("="*60)

device = torch.device(config.DEVICE)

# Use best fold model for test
best_fold_idx = np.argmax(val_aucs)
best_fold_result = fold_results[best_fold_idx]

print(f"\nUsing model from Fold {best_fold_idx + 1} (best val AUC: {best_fold_result['val_auc']:.4f})")

# Reinitialize mastery for test
mastery_matrix = torch.full(
    (num_students, num_kcs),
    config.MASTERY_INIT,
    dtype=torch.float32,
    device=device
)

# Compute features with mastery
node_features = compute_node_features_with_mastery(
    df, num_students, num_questions, num_steps, num_kcs,
    mastery_matrix
).to(device)

# Build full graph
edge_index_dict, all_relations = build_heterogeneous_graph(
    df, num_students, num_questions, num_steps, num_kcs
)

# Load best model
model = RGCNWithDualPropagation(
    input_dim=5 + num_kcs,
    hidden_dim=config.HIDDEN_DIM,
    num_relations=len(all_relations),
    num_kcs=num_kcs
).to(device)

model.load_state_dict(best_fold_result['model_state'])

# Create test loader
test_loader = create_data_loader(df_test, config.BATCH_SIZE, shuffle=False)

print(f"\nTest set: {len(test_students)} students, {len(df_test):,} interactions")
print("Evaluating...")

# Evaluate on test set
test_auc, test_acc = evaluate(
    model, test_loader, mastery_matrix, node_features, edge_index_dict,
    all_relations, num_students, num_questions, num_steps, num_kcs, device
)

print(f"\n{'='*60}")
print(f"TEST SET RESULTS")
print(f"{'='*60}")
print(f"Test AUC: {test_auc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"{'='*60}")

# Store final results
final_results = {
    'cv_mean_auc': cv_results['mean_val_auc'],
    'cv_std_auc': cv_results['std_val_auc'],
    'test_auc': test_auc,
    'test_acc': test_acc,
    'num_test_students': len(test_students),
    'num_test_interactions': len(df_test),
    'total_params': cv_results['total_params']
}

print("\n✅ Test evaluation complete!")

In [ ]:
# ============================================================
# CELL 8: FINAL RESULTS DISPLAY
# ============================================================

print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)

print("\n╔══════════════════════════════════════════════════════════════╗")
print("║       GraphKT M3 Final (Features + Weights + Dual Prop)      ║")
print("║                    Algebra 2005-2006 Dataset                 ║")
print("╠══════════════════════════════════════════════════════════════╣")
print("║                                                              ║")
print("║  5-Fold CV Validation:                                       ║")
print(f"║    AUC:      {final_results['cv_mean_auc']:.4f} ± {final_results['cv_std_auc']:.4f}                              ║")
print(f"║    Accuracy: {cv_results['mean_val_acc']:.4f} ± {np.std(val_accs):.4f}                              ║")
print("║                                                              ║")
print(f"║  Test Set (held-out, {len(test_students)} students):                          ║")
print(f"║    AUC:      {final_results['test_auc']:.4f}                                        ║")
print(f"║    Accuracy: {final_results['test_acc']:.4f}                                        ║")
print("║                                                              ║")
print(f"║  Model: {final_results['total_params']:,} parameters                              ║")
print(f"║  Training: {int(np.mean(epochs_list))} epochs (CV average)                           ║")
print("║  Split: Student-level (Split B), no leakage                  ║")
print("║                                                              ║")
print("║  Architecture:                                               ║")
print("║    • Student features: [5D + 397D mastery] = 402D            ║")
print("║    • Dual propagation (Structural + Knowledge)               ║")
print("║    • Mastery in both features AND weights                    ║")
print("║    • Dynamic feature updates each epoch                      ║")
print("║                                                              ║")
print("╚══════════════════════════════════════════════════════════════╝")

print("\nTable 2: Model Comparison (for paper)")
print("-" * 65)
print(f"{'Model':<20} {'Val AUC':<16} {'Test AUC':<12} {'Params':<12}")
print("-" * 65)
print(f"{'GraphKT M3 Final':<20} "
      f"{final_results['cv_mean_auc']:.4f} ± {final_results['cv_std_auc']:.4f}   "
      f"{final_results['test_auc']:.4f}       "
      f"{final_results['total_params']:,}")
print("-" * 65)

# Comparison with previous models (if available)
print("\n📊 Expected Comparison:")
print("  M1 (GCN):       ~76.17% val, ~72.55% test")
print("  M2 (RGCN):      ~76.11% val, ~72.30% test")
print("  M3a (Features): ~73.61% val, ~74.66% test")
print("  M3b (Weights):  ~76.14% val, ~75.43% test")
print(f"  M3 Final:       {final_results['cv_mean_auc']*100:.2f}% val, {final_results['test_auc']*100:.2f}% test ⭐")

# Calculate improvements
if final_results['test_auc'] > 0.7543:  # vs M3b
    improvement = (final_results['test_auc'] - 0.7543) * 100
    print(f"\n🎉 Improvement over M3b: +{improvement:.2f}% test AUC")
    print("   ✅ Synergy between features and weights confirmed!")

print("\n" + "="*60)
print("✅ M3 FINAL TRAINING COMPLETE!")
print("="*60)
print("\nNext steps:")
print("1. Compare with M1/M2/M3a/M3b results")
print("2. Run hyperparameter optimization (optional)")
print("3. Test on additional datasets (ASSIST2009)")
print("4. Prepare final comparison table for paper")